In [1]:
import pandas as pd

In [2]:
output_directory = "../data"
rerun = False

Read in csv to Python

In [3]:
if rerun:
    df = pd.read_csv(f"{output_directory}/combined_data.csv",index_col=0, parse_dates=True)

Save to parquet (as one file)

In [4]:
%%time

if rerun:
    df.to_parquet(f"{output_directory}/combined_data.parquet")

CPU times: total: 0 ns
Wall time: 0 ns


Save to parquet (partition by model)

In [5]:
%%time

if rerun:
    df.to_parquet(f"{output_directory}/combined_data_partition.parquet",partition_cols=['model'])

CPU times: total: 0 ns
Wall time: 0 ns


Compare file sizes:

In [6]:
%%sh
du -sh "../data/combined_data.csv"
du -sh "../data/combined_data.parquet"
du -sh "../data/combined_data_partition.parquet"

5.7G	../data/combined_data.csv
542M	../data/combined_data.parquet
550M	../data/combined_data_partition.parquet


Time Python read parquet file and count:

In [7]:
%%time

df = pd.read_parquet(f"{output_directory}/combined_data.parquet")
print(df["model"].value_counts())

MPI-ESM1-2-HR       5154240
TaiESM1             3541230
NorESM2-MM          3541230
CMCC-CM2-HR4        3541230
CMCC-CM2-SR5        3541230
CMCC-ESM2           3541230
SAM0-UNICON         3541153
FGOALS-f3-L         3219300
GFDL-CM4            3219300
GFDL-ESM4           3219300
EC-Earth3-Veg-LR    3037320
MRI-ESM2-0          3037320
BCC-CSM2-MR         3035340
MIROC6              2070900
ACCESS-CM2          1932840
ACCESS-ESM1-5       1610700
INM-CM5-0           1609650
INM-CM4-8           1609650
KIOST-ESM           1287720
FGOALS-g3           1287720
MPI-ESM1-2-LR        966420
NESM3                966420
AWI-ESM-1-1-LR       966420
MPI-ESM-1-2-HAM      966420
NorESM2-LM           919800
BCC-ESM1             551880
CanESM5              551880
Name: model, dtype: int64
CPU times: total: 19.8 s
Wall time: 7.39 s


Time R read parquet file and count:

In [8]:
%load_ext rpy2.ipython

C:\ProgramData\Miniconda3\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [9]:
%%time
%%R

suppressPackageStartupMessages(library(dplyr,quietly=TRUE))
output_directory <- "../data"
#df <- arrow::read_parquet(paste0(output_directory,"/","combined_data_partition.parquet"))
df <- arrow::open_dataset(paste0(output_directory,"/","combined_data_partition.parquet"), 
                          format="parquet", 
                          partitioning=c("model"))
df %>%
    group_by(model) %>%
    summarize(cnt=n()) %>%
    ungroup() %>%
    collect()

# A tibble: 27 x 2
   model                cnt
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CMCC-CM2-HR4     3541230
 7 CMCC-CM2-SR5     3541230
 8 CMCC-ESM2        3541230
 9 EC-Earth3-Veg-LR 3037320
10 CanESM5           551880
# ... with 17 more rows
CPU times: total: 2.17 s
Wall time: 2.18 s


## Discussion

- Pandas build in function can write/read to parquet files easily (R with the arrow package)
- Parquet file type significantly reduce size in storage (duh) (5.7GB shrink to 542M without partitioning and 550M with partition)
    - No partition = slightly smaller in total size
    - Partition = slightly more in total size but probably best for I/O since you can choose easily what to read in
- Python took significantly more time to read and count rows than R
    - could be a pandas vs arrow optimization problem